In [1]:
import numpy as np

from chugunov_indicator import skip_chugunov_2009

import pynucastro as pyna
import yt

In [2]:
# Double Detonation data: http://groot.astro.sunysb.edu/common/
ds = yt.load('./data/subch_plt17526/')
ad = ds.all_data()

yt : [INFO     ] 2024-10-10 01:17:10,147 Parameters: current_time              = 0.800001438968278
yt : [INFO     ] 2024-10-10 01:17:10,148 Parameters: domain_dimensions         = [ 640 1280    1]
yt : [INFO     ] 2024-10-10 01:17:10,148 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-10-10 01:17:10,149 Parameters: domain_right_edge         = [5.12000000e+09 1.02400000e+10 6.28318531e+00]


In [3]:
reaclib_library = pyna.ReacLibLibrary()

nuclei = [field[2:-1] for field in np.array(ds.field_list)[:,1] if "X(" in field]
comp = pyna.Composition(nuclei)
comp.set_solar_like()

mynet = reaclib_library.linking_nuclei(comp.keys())
pynet = pyna.PythonNetwork(libraries=[mynet])

screen_map = pyna.screening.get_screening_map(
    pynet.get_rates(),
    symmetric_screening=pynet.symmetric_screening
)

In [4]:
pair = screen_map[5]
scn_fac = pyna.make_screen_factors(pair.n1, pair.n2)

First, we consider a case where screening can't be skipped.

In [5]:
plasma = pyna.make_plasma_state(2e7, 1e4, comp.get_molar())
pyna.screening.chugunov_2009(plasma, scn_fac)

13.611971657871864

In [6]:
%%timeit

F = pyna.screening.chugunov_2009(plasma, scn_fac)

1.7 µs ± 39.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
%%timeit

skip = skip_chugunov_2009(plasma, scn_fac)

1.25 µs ± 29.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Next, we consider a case where it can be skipped.

In [8]:
plasma = pyna.make_plasma_state(1e9, 1e2, comp.get_molar())
pyna.screening.chugunov_2009(plasma, scn_fac)

1.0012163585951908

In [9]:
%%timeit

F = pyna.screening.chugunov_2009(plasma, scn_fac)

1.67 µs ± 46.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [10]:
%%timeit

skip = skip_chugunov_2009(plasma, scn_fac)

1.26 µs ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


This means that `skip_chugunov_2009` is around 25% faster in all cases.

To test how effective this, we can test more double detonation data.

In [11]:
# Double Detonation data: http://groot.astro.sunysb.edu/common/
ds = yt.load('./data/subch_plt21866/')
ad = ds.all_data()

yt : [INFO     ] 2024-10-10 01:18:07,888 Parameters: current_time              = 1.00002871903213
yt : [INFO     ] 2024-10-10 01:18:07,889 Parameters: domain_dimensions         = [ 640 1280    1]
yt : [INFO     ] 2024-10-10 01:18:07,889 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-10-10 01:18:07,891 Parameters: domain_right_edge         = [5.12000000e+09 1.02400000e+10 6.28318531e+00]


In [12]:
Xs = (np.array([ad[f"X({nucleus})"] for nucleus in nuclei]).T)
T, D = ad["Temp"], ad["density"]

In [13]:
rng = np.random.default_rng(10)

total = 10000
choices = rng.choice(np.arange(len(ad["density"])), total)
choices

array([3123643, 3845543, 1062025, ...,  929823,  784546,  171927])

In [14]:
plasma_states = []
for i in choices:
    comp.set_array(Xs[i])
    plasma = pyna.make_plasma_state(T[i], D[i], comp.get_molar())
    plasma_states.append(plasma)

In [15]:
%%timeit

for plasma_ in plasma_states:
    F = pyna.screening.chugunov_2009(plasma_, scn_fac)

18.4 ms ± 712 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit

for plasma_ in plasma_states:
    if skip_chugunov_2009(plasma, scn_fac):
        F = 1
    else:
        F = pyna.screening.chugunov_2009(plasma_, scn_fac)

13.1 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As such, this can improve screening calculation times by around $25\%$ when used with `pynucastro`.

Note that since these values are the same for given `PlasmaState` and `ScreenFactors` objects, the value $C_{PS} := C_0 + C_1 + C_2$ can be pre-computed when the `PlasmaState` object is initialized, and same with $C_{SF} := C_3$ for the `ScreenFactors` object. Since the same `PlasmaState` is used for several screening calculations, this would speed up the calculation times even further.